<a href="https://colab.research.google.com/github/Bluseass/Telegram-spam-ham/blob/main/Telegram_spam_ham_96_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path = pd.read_csv('/content/drive/MyDrive/telegram.csv')
df = pd.DataFrame(path)

In [ ]:
import string

def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table).lower() for word in tokens if word.isalpha()]
    return tokens

In [ ]:
df['clean_tokens'] = df['text'].apply(clean_doc)

In [ ]:
import pandas as pd
from random import randint
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

df = pd.read_csv('/content/drive/MyDrive/telegram.csv')
lines = df['text'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
vocab_size = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(lines)
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

X = sequences[:, :-1]
y = sequences[:, -1]

model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=2)

def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = []
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')
        yhat = model.predict(encoded, verbose=0)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat.argmax():
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

seed_text = lines[randint(0, len(lines))]
print(seed_text + '\n')

generated = generate_seq(model, tokenizer, max_length-1, seed_text, 50)
print(generated)